# Wykład 4:
# W wyjątkowej sieci z plikami
##  wyjątki, praca z plikami, contextmanager
# Wojciech Łuszczyński
### 20.XI.2018, Daft Academy Python4Beginners

# 1. Exceptions

## Syntax Errors vs Exceptions?

### Syntax Errors

In [1]:
for i in range(10) print('HELLO')

SyntaxError: invalid syntax (<ipython-input-1-7234bef2bf69>, line 1)

## Syntax Error
- błąd interpretera podczasu parsowania skryptu
- błąd składniowy
- python nie jest w stanie sparsować naszego kodu
- domyślnie pythonowy intepreter poda linijkę i miejsce wystąpienia błędu składniowego i przestanie dalej przetwarzać plik

https://docs.python.org/3.7/tutorial/errors.html#syntax-errors

### Exceptions

In [2]:
f = open('test.txt', 'w')
f.write('Linia pierwsza\n')
f.write('Linia druga\n')
f.write('Linia trzecia\n')
x = 1 / 0
f.close()

ZeroDivisionError: division by zero

### Exceptions
- błędy w trakcie wykonywania programu
- zazwyczaj zwracają jakiś opis błędu
- można sobie z nimi radzić


- https://docs.python.org/3.7/tutorial/errors.html#exceptions
- https://jeffknupp.com/blog/2013/02/06/write-cleaner-python-use-exceptions/

## "Łapanie" wyjątków

### Try Except

In [3]:
try:
    my_int = int('4')
    print(my_int)
except ValueError:
    print('Złapałem value error!')

4


- Na początek wykonywany jest kod pomiędzy try, a except
- Jeżeli nie ma błędu, kod spomiędzy try a except jest wykonany w całości, a kod w except jest pomijany

In [4]:
try:
    my_int = int('nie int :(')
    print(my_int)
except ValueError:
    print('Złapałem value error!')

Złapałem value error!


- Jeżeli pojawi się błąd, wykonanie kodu spomiędzy try a except jest przerwane
- Jeżeli błąd jest łapanego przez nas typu, to wykonywany jest kod w except

In [5]:
try:
    my_int = int('4')
    print(my_int)
    a = {'b': 'c'}
    print(a['d'])
except (ValueError, KeyError):  # te nawiasy są ważne!
    print('Dostalem ValueError lub KeyError')


4
Dostalem ValueError lub KeyError


- Można łapać więcej niż jeden wyjątek jednocześnie

In [6]:
try:
    my_int = int('4')
    print(my_int)
    a = {'b': 'c'}
    print(a['d'])
except ValueError:
    print('Złapałem ValueError')
except KeyError:
    print('Złapałem KeyError')

4
Złapałem KeyError


- Blok Try Except może przyjąc tak dużo Except ile potrzebujemy

In [7]:
try:
    my_int = int('4')
    print(my_int)
    a = {'b': 'c'}
    print(a.append('my_int'))
except ValueError:
    print('Złapałem ValueError')
except KeyError:
    print('Złapałem KeyError')    

4


AttributeError: 'dict' object has no attribute 'append'

- Jeżeli wystąpi inny błąd - wyjątek nie jest łapany i program kończy się z takim błędem

In [8]:
def fun(my_int, my_dict):
    print (my_int, my_dict)
    try:
        int(my_int)
        my_dict['a']
    except:
        print('Złapałem dowolny błąd')

fun('xx', {'a': 'b'})
print('******')
fun('4', {'c': 'd'})

xx {'a': 'b'}
Złapałem dowolny błąd
******
4 {'c': 'd'}
Złapałem dowolny błąd


- Możliwe jest łapanie dowolnego rodzaju błędu - używać z rozwagą - zazwyczaj chcemy łapać konkretny błąd.
- W ten sposób można łatwo przegapić zwykły błąd programistyczny jak staramy się obsługiwać wszystkie błędy na raz

### Wbudowane wyjątki

- https://docs.python.org/3.7/library/exceptions.html#bltin-exceptions

- wszystkie klasy wyjątków dziedziczą pośrednio lub bezpośrednio z BaseException
- w większości przypadków instancje klasy wyjątków dziedziczą z Exception (sprawdź w dokumentacji)
- rozbudowana hierarchia błędów - można się uczyć z czasem ;)
- warto zobaczyć: https://docs.python.org/3.7/library/exceptions.html#exception-hierarchy

## Rzucanie wyjątkiem

In [9]:
raise ValueError('Poważny błąd!')

ValueError: Poważny błąd!

- Możemy rzucać zarówno instancję wyjątku, jak i klasę

In [10]:
raise ValueError

ValueError: 

In [11]:
def fun(my_int, my_dict):
    print (my_int, my_dict)
    try:
        int(my_int)
        my_dict['a']
    except ValueError:
        print('Złapałem ValueError')
        raise

fun('xx', {'a': 'b'})

xx {'a': 'b'}
Złapałem ValueError


ValueError: invalid literal for int() with base 10: 'xx'

- powyższy przykład pokazuje że możemy obsłużyć wyjątek i nadal go "rzucić" by np inna część kodu go obsłużyła

## Własna definicja klasy wyjątku

Definiowanie własnych błędów
- błąd powinien dziedziczyć po Exception
- zazwyczaj klasy błędów są proste
- często, gdy planujemy stworzyć więcej niż jeden błąd, tworzy się wspólną nadklasę dla modułu

In [12]:
class IncorrectGuessError(Exception):
    def __init__(self, difference):
        self.difference = difference

class NumberTooSmall(IncorrectGuessError):
    pass

class NumberTooBig(IncorrectGuessError):
    pass

def guess_number(guess):
    number = 10
    if guess > number:
        raise NumberToBig(guess - number)
    elif guess < number:
        raise NumberTooSmall(number - guess)
    else:
        print('Brawo')
try:
    guess_number(4)
except NumberTooSmall as exc:
    print('Za malo o {}'.format(exc.difference))
except NumberTooBig:
    print('Za duzo o {}'.format(exc.difference))

Za malo o 6


Dzięki instrukcji `else` możemy wykonać jakiś kod TYLKO wtedy, gdy kod w bloku `try` wykona się bez błędu

In [13]:
def fun(my_int, my_dict):
    print (my_int, my_dict)
    try:
        int(my_int)
        my_dict['a']
    except ValueError:
        print('Złapałem błąd')
    else:
        print('Bez błędu!')
fun('4', {'a': 'b'})
fun('x', {'c': 'd'})
fun('4', {'c': 'd'})

4 {'a': 'b'}
Bez błędu!
x {'c': 'd'}
Złapałem błąd
4 {'c': 'd'}


KeyError: 'a'

Kod, który chcemy żeby wykonał się zawsze umieszczamy w bloku `finally`

In [14]:
def fun(my_int, my_dict):
    print (my_int, my_dict)
    try:
        int(my_int)
        my_dict['a']
    except ValueError:
        print('Złapałem błąd')
    else:
        print('Bez błędu!')
    finally:
        print('*' * 80)
fun('4', {'a': 'b'})
fun('x', {'c': 'd'})
fun('4', {'c': 'd'})

4 {'a': 'b'}
Bez błędu!
********************************************************************************
x {'c': 'd'}
Złapałem błąd
********************************************************************************
4 {'c': 'd'}
********************************************************************************


KeyError: 'a'

kod w bloku `finally` wykonuje się zawsze
- nie zależnie od tego czy wyjątek był obsłużony czy nie
- wykona się nawet jeśli w `try`, `except`, `else` nastąpi `return` lub `break`
- jeśli wyjątek nie jest obsłużony `finally` po wykonaniu kodu w swoim bloku rzuci ponownie tym samym wyjątkiem

# 2. Pliki

In [15]:
f = open('test.txt', 'w')
f.write('Linia pierwsza\n')
f.write('Linia druga\n')
f.write('Linia trzecia\n')
f.close()

## Otwieranie pliku:
- przyjmuje relatywną lub absolutną ścieżkę do pliku który otwieramy
- reszta argumentów jest opcjonalna, często używa się argumentu mode
- mode określa w jakim trybie otwieramy plik
- dostępne tryby: 'r' (read), 'w' (write), 'a' (append), 'r+' (read+write)
- domyślny tryb to 'r' (read)
- write nadpisuje plik, append dodaje do końca
- mode określa dodatkowo tryb tekstowy ('t' - domyślnie) lub binarny ('b')
- zwraca obiekt pliku
- domyślne kodowanie tekstu w trybie tekstowym to UTF-8

## Przydatne linki

- https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files
- https://docs.python.org/3/library/functions.html#open
- https://docs.python.org/3/glossary.html#term-file-object


## Zamykanie pliku:
- metoda close dostępna jest na obiekcie pliku
- CPython dzięki garbage collectorowi z licznikiem referencji sam zadba o zamknięcie pliku
- mimo tego powinniśmy ZAWSZE zadbać o to sami - to, że CPython zamknie plik to szczegół tej implementacji, część innych implementacji tego nie zapewnia

## Dlaczego nie można zostawić otwartego pliku?
- wpisane przez nas zmiany mogą nie zostać sflushowane na dysk (zależne od implementacji - flush zapewnia, że dane z bufora zapisują się na dysku)
- systemy operacyjne mają limit otwartych plików - możemy dostać wyjątek w przypadku otwarcia zbyt dużej liczby plików
- czasami niezamknięty plik uniemożliwia odmontowanie, wyjęcie jakiegoś nośnika danych


### Przydatne linki:
- https://stackoverflow.com/questions/3167494/how-often-does-python-flush-to-a-file
- https://askubuntu.com/questions/701491/is-closing-a-file-after-having-opened-it-with-open-required-in-python

In [16]:
f = open('test.txt', 'w')
f.write('Linia pierwsza\n')
f.write('Linia druga\n')
f.write('Linia trzecia\n')
x = 1 / 0
f.close()

ZeroDivisionError: division by zero

In [17]:
try:
    f = open('test.txt', 'w')
    f.write('Linia pierwsza\n')
    f.write('Linia druga\n')
    f.write('Linia trzecia\n')
    x = 1 / 0
finally:
    print('Zamykam plik')
    f.close()
    print('Plik zamkniety')

Zamykam plik
Plik zamkniety


ZeroDivisionError: division by zero

O zamknięcie pliku da się zadbać łatwiej

In [ ]:
with open('test.txt', 'w') as f:
    f.write('Linia pierwsza\n')
    f.write('Linia druga\n')
    f.write('Linia trzecia\n')
    x = 1 / 0

# 3. Context Manager

## contextmanager

- obiekt, który definiuje w jakim kontekście ma się wywołać owijany kod
- określa, co ma się wykonać przed i po wykonaniu bloku kodu owijanego
- zazwyczaj wywoływane ze pomocą `with`, ale da się też bezpośrednio wywołać poszczególne metody
- przykłady użycia: zamykanie otwartych plików, zwalnianie zasobów/locków, przywracanie jakeigoś globalnego stanu


- https://docs.python.org/3/reference/datamodel.html#context-managers
- https://docs.python.org/3/library/stdtypes.html#typecontextmanager

## with

- `with` jest używane do owinięcia bloku kodu za pomocą metod określonych w context managerze
- używane do zarządzania zasobami w danym bloku
- da się dzięki temu ładnie owinąć `try`...`except`...`finally`


- https://docs.python.org/3/reference/compound_stmts.html#with
- https://www.python.org/dev/peps/pep-0343/

In [18]:
with open('test.txt', 'w') as f:
    f.write('Linia pierwsza\n')
    f.write('Linia druga\n')
f.write('Linia trzecia\n')

ValueError: I/O operation on closed file.

# 4. Pracy z plikami ciąg dalszy

# Rozwiążmy zadanie rekrutacyjne w pythonie

Jesteś pracownikiem firmy ConfWorld.
Twoja firma zajmuje się organizowaniem konferencji na całym świecie.
Dostałes plik z danymi (CSV) o przeszłych konferencjach powiązanych z pewnymi językami programowania.
W pierwszym wierszu, w każdej kolumnie masz podany numer konferencji i nazwę języka programowania, którego dotyczyła.
W kolejnych wierszach w każdej kolumnie masz podanych uczestników każdej konferencji wraz z zyskiem jaki przynieśli firmie ConfWorld.

Wiesz, że każdy uczestnik ma adres email w formie:
- `aaaaaa@bbbbb.cc`
- gdzie:
- `aaaaaa` - login
- `bbbbb` - domena firmy w jakiej pracuje uczestnik
- `cc` - kraj zamieszkania uczestnika
- długość pól a, b, nie jest stała.

maile zapisaną są w formacie takim że:
- `ala@xyz.pl` i `ala@xyz.at` reprezentują dwie różne osoby.

Z powodów takich jak różne kanały sprzedaży, termin zakupu wejściówek, przewalutowań i innych prawie każdy uczestnik danej konferencji przyniósł firmie ConfWorld inny zysk.

### 1. O którym języku programowania zorganizowano najwiecej konferencji?
### 2. Ile różnych osób uczestniczyło w sumie we wszystkich konferencjach o języku Python?
### 3. Który język programowania okazał sie najbardziej dochodowy?
### 4. Mieszkańcy, którego kraju zapłacili najwięcej za wstęp na wszystkie konferencje?

In [19]:
from collections import defaultdict

def get_data(filename):
    rows = list()
    dane = dict()
    with open(filename, "r") as f:
        # read headers and make them usefull for future use
        headers = f.readline().strip('\n').split(";")
        for i in range(0, len(headers)):
            # example: "konferencja 0 (Ruby)" -> ('0', 'ruby')
            headers[i] = tuple(headers[i].lower().strip(') ').replace('(', '').split(' ')[1:])
        # read all lines
        for line in f:
            rows.append(line.strip(') \n').split(";"))
        # change CSV lines representation into columns of data per header
        for i in range(0, len(headers)):
            # columns are not equally filled with data so filter out empty cells
            dane[headers[i]] = list(filter(lambda g: len(g), [y[i] for y in rows]))
    return dane
data = get_data("../zadanie_rekrutacyjne/dane.csv")

In [20]:
# 1. O którym języku programowania zorganizowano najwiecej konferencji?

def task_1(data):
    headers = [x[1] for x in data.keys()]
    langs = defaultdict(int)
    for x in headers:
        langs[x] = headers.count(x)
    return max(langs.keys(), key=(lambda key: langs[key]))
print("1:", task_1(data))


1: java


In [21]:
# 2. Ile różnych osób uczestniczyło w sumie we wszystkich konferencjach o języku Python?

def task_2(data):
    # adding to set wont create duplicates
    python_participants = set()
    for key, val in data.items():
        if 'python' == key[1]:
            for pp in val:
                python_participants.add(pp.split('/')[0])

    return len(python_participants)
print("2:", task_2(data))

2: 1868


In [22]:
# 3. Który język programowania okazał sie najbardziej dochodowy?

def task_3(data):
    # defaultdict(int) is initializes with 0
    revenue = defaultdict(int)
    for key, val in data.items():
        x = key[1]
        for y in val:
            revenue[x] += int(y.split('/')[1]) 
    
    # nice way to get MAX from dict, just implement the KEY getter
    return max(revenue.keys(), key=(lambda key: revenue[key]))

print("3:", task_3(data))

3: java


In [23]:
# 4. Mieszkańcy, którego kraju zapłacili najwięcej za wstęp na wszystkie konferencje?

def task_5(data):
    countries = defaultdict(int)
    for val in data.values():
        for y in val:
            mail, greens = y.split('/')
            country = mail.split('@')[1].split('.')[1]
            countries[country] += int(greens)
    return max(countries.keys(), key=(lambda key: countries[key]))

print("5:", task_5(data))

5: sm


# 5. Praca z plikami w trybie binarnym

In [24]:
with open('p4b_logo.bmp', 'rb') as fr:
    data = fr.read()
    print(type(data))
with open('copy_p4b_logo.bmp', 'wb') as fw:
    fw.write(data)

<class 'bytes'>


- tryb otwarcia pliku z 'b' na końcu
- umożliwia pracę z plikami niezawierającymi tekstu

Przydatne funkcje
- seek - ustawia aktualną pozycję wskaźnika pozycji w pliku na wybraną
- tell - zwraca aktualną pozycję wskaźnika w pliku

In [25]:
with open('p4b_logo.bmp', 'rb') as fr:
    print(fr.tell())
    ten_bytes = fr.read(10)
    print(fr.tell())
    fr.seek(0)
    print(fr.tell())
    fr.seek(100)
    print(fr.tell())

0
10
0
100


## bytes
- wczytywane dane binarne są typu bytes - niezmiennej sekwencji 8-bitowych wartości (0-255)


## bytearray
- zmienna (mutowalna) wersja bytes
- jeżeli chcemy zmodyfikować plik, lepiej używać bytearray


- https://www.w3resource.com/python/python-bytes.php#bytes
- https://www.devdungeon.com/content/working-binary-data-python

In [26]:
with open('p4b_logo.bmp', 'rb') as fr:
    data = fr.read()
    print(type(data))
    data = bytearray(data)
    print(type(data))
with open('another_copy_p4b_logo.bmp', 'wb') as fw:
    fw.write(data)

<class 'bytes'>
<class 'bytearray'>
